# Collision Avoidance - Live Demo (Resnet18)

In this notebook we'll use the model we trained to detect whether the robot is ``free`` or ``blocked`` to enable a collision avoidance behavior on the robot.  

## Load the trained model

We'll assumed that you've already downloaded the ``best_model.pth`` to your workstation as instructed in the training notebook.  Now, you should upload this model into this notebook's
directory by using the Jupyter Lab upload tool.  Once that's finished there should be a file named ``best_model.pth`` in this notebook's directory.  

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

In [1]:
import torch
import torchvision

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

Next, load the trained weights from the ``best_model_resnet18.pth`` file that you uploaded

In [2]:
model.load_state_dict(torch.load('collision_avoider.pth'))

<All keys matched successfully>

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [3]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesn't *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Great! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that the robot is blocked.  We'll also display a slider that allows us to control the robot's base speed.

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=2)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.VBox([widgets.HBox([image, blocked_slider]), speed_slider]))

We'll also create our robot instance which we'll need to drive the motors.

In [6]:
from jetbot import Robot

robot = Robot()
robot.stop()

In [7]:
def turn_left_90():
    robot.left(0.4)
    time.sleep(.18)
    robot.stop()

Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

In [8]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)

    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.8:
        robot.left_motor.value = 0.212
        robot.right_motor.value = 0.2
    else:
        turn_left_90()
        robot.left_motor.value = 0.212
        robot.right_motor.value = 0.2
        flag = True
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to initialize

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing. 

We accomplish that with the ``observe`` function.

> WARNING: This code may move the robot!! Adjust the speed slider we defined earlier to control the base robot speed.  Some kits can move fast, so start slow, and gradually increase the value.

In [13]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In [10]:
def go_on(change):
    camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame.  Perhaps start by placing your robot on the ground and seeing what it does when it reaches an obstacle.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [11]:
import time

def pause(change):
    camera.unobserve(update, names='value')

    time.sleep(0.1)  # add a small sleep to make| sure frames have finished process}ing

    robot.stop()


In [12]:
camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make| sure frames have finished process}ing

robot.stop()

### Interrupt part, simulated by manually control for test

device recognizer init

In [14]:
from device_predictor import DevicePredictor
dp = DevicePredictor()

In [15]:
def call_predict(change):
    print(dp.predict(camera.value))

In [16]:
def stop(change):
    robot.stop()
    
def step_forward(change):
    #robot.forward(0.4)
    robot.left_motor.value = 0.415
    robot.right_motor.value = 0.4
    #robot.right(0.4)
    #time.sleep(0.5)
    #robot.stop()

def step_backward(change):
    robot.backward(0.4)
    time.sleep(0.5)
    robot.stop()

def step_left(change):
    robot.left(0.15)

def step_right(change):
    robot.right(0.15)

In [17]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# mode change buttons
man_button = widgets.Button(description='interrupt', layout=button_layout)
take_pic_button = widgets.Button(description='predict', layout=button_layout)
auto_button = widgets.Button(description='return', layout=button_layout)

mode_box = widgets.HBox([man_button, take_pic_button, auto_button], layout=widgets.Layout(align_self='center'))

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button, mode_box])
display(controls_box)

In [18]:
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)
man_button.on_click(pause)
take_pic_button.on_click(call_predict)
auto_button.on_click(go_on)

### ------------------------------------------------------------------------------------------------

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

To continue streaming call the following.

In [ ]:
camera_link.link()  # stream to browser (wont run camera)

Again, let's close the camera conneciton properly so that we can use the camera in other notebooks.

In [19]:
camera.stop()